# 이화에 바란다 청원통과 확률 예측

## 1. 데이터 로딩

In [85]:
import pandas as pd

# 24년부터 19년까지의 이화에 바란다 게시글 텍스트 데이터
petition_df = pd.read_csv('./24_19_2_ewha.csv', sep=",") 
petition_df.head(10)

,NO,제목,공감/비공감,조회수,등록일,카테고리,세분화
0,7803,북한학과 박사 논문 연구계획서 불합격 제도 시정 요구,0 / 0,72,2024.03.06,대학,NaN
1,7802,IT 사설강의 '인프런'과 대학 제휴 협약을 맺어주세요,28 / 0,172,2024.02.13,학생지원,학술지원
2,7801,학점 포기제 일부 허용 건의,0 / 0,9,2024.01.29,행정지원,학교관련 제도
3,7800,편입학 제도 개선 관련 문의,0 / 0,2,2024.01.23,행정지원,학교관련 제도
4,7799,기숙사 통금 시간 연장 건의,286 / 2,1013,2024.01.19,생활지원,기숙사
5,7798,계리사 합격장려금 지원 건의,27 / 0,156,2024.01.11,학생지원,고시지원
6,7797,본교에서도 통합과정의 박사과정생 또는 박사수료자가 박사학위논문을 청구하기 직전까지 ...,21 / 0,157,2024.01.09,행정지원,학교관련 제도
7,7796,"18학년도 입학자 입니다 스페인어 2를 들어야 졸업할 수 있는데, 이제는 계절로도 ...",0 / 0,89,2024.01.03,대학,입학 및 학사운영
8,7795,마이유레카 모바일 웹 버전 개선 부탁드립니다,4 / 0,100,2023.12.05,기타,NaN
9,7794,법정의무교육 수강에 대한 제도 구축을 건의합니다,2 / 0,64,2023.12.01,행정지원,학교관련 제도


- 비공감, 공감 열 생성

In [86]:
# '공감/비공감' 피처를 '/' 기준으로 나누어 '공감', '비공감' 열 생성
petition_df[['공감', '비공감']] = petition_df['공감/비공감'].str.split('/', expand=True)

In [87]:
# '공감/비공감' 피처 제거
petition_df.drop('공감/비공감', axis=1, inplace=True)
petition_df.head(10)

,NO,제목,조회수,등록일,카테고리,세분화,공감,비공감
0,7803,북한학과 박사 논문 연구계획서 불합격 제도 시정 요구,72,2024.03.06,대학,NaN,0,0
1,7802,IT 사설강의 '인프런'과 대학 제휴 협약을 맺어주세요,172,2024.02.13,학생지원,학술지원,28,0
2,7801,학점 포기제 일부 허용 건의,9,2024.01.29,행정지원,학교관련 제도,0,0
3,7800,편입학 제도 개선 관련 문의,2,2024.01.23,행정지원,학교관련 제도,0,0
4,7799,기숙사 통금 시간 연장 건의,1013,2024.01.19,생활지원,기숙사,286,2
5,7798,계리사 합격장려금 지원 건의,156,2024.01.11,학생지원,고시지원,27,0
6,7797,본교에서도 통합과정의 박사과정생 또는 박사수료자가 박사학위논문을 청구하기 직전까지 ...,157,2024.01.09,행정지원,학교관련 제도,21,0
7,7796,"18학년도 입학자 입니다 스페인어 2를 들어야 졸업할 수 있는데, 이제는 계절로도 ...",89,2024.01.03,대학,입학 및 학사운영,0,0
8,7795,마이유레카 모바일 웹 버전 개선 부탁드립니다,100,2023.12.05,기타,NaN,4,0
9,7794,법정의무교육 수강에 대한 제도 구축을 건의합니다,64,2023.12.01,행정지원,학교관련 제도,2,0


- 카테고리 피처와 세분화 피처 합한 카테고리 피처 반환

In [88]:
# '카테고리2' 열을 추가, 세분화 값이 NaN이 아닐 때만 값을 합침
petition_df['카테고리'] = petition_df.apply(lambda x: x['카테고리'] + " - " + x['세분화'] if pd.notnull(x['세분화']) else x['카테고리'], axis=1)

In [89]:
# '공감/비공감' 피처 제거
petition_df.drop('세분화', axis=1, inplace=True)
petition_df.head(10)

,NO,제목,조회수,등록일,카테고리,공감,비공감
0,7803,북한학과 박사 논문 연구계획서 불합격 제도 시정 요구,72,2024.03.06,대학,0,0
1,7802,IT 사설강의 '인프런'과 대학 제휴 협약을 맺어주세요,172,2024.02.13,학생지원 - 학술지원,28,0
2,7801,학점 포기제 일부 허용 건의,9,2024.01.29,행정지원 - 학교관련 제도,0,0
3,7800,편입학 제도 개선 관련 문의,2,2024.01.23,행정지원 - 학교관련 제도,0,0
4,7799,기숙사 통금 시간 연장 건의,1013,2024.01.19,생활지원 - 기숙사,286,2
5,7798,계리사 합격장려금 지원 건의,156,2024.01.11,학생지원 - 고시지원,27,0
6,7797,본교에서도 통합과정의 박사과정생 또는 박사수료자가 박사학위논문을 청구하기 직전까지 ...,157,2024.01.09,행정지원 - 학교관련 제도,21,0
7,7796,"18학년도 입학자 입니다 스페인어 2를 들어야 졸업할 수 있는데, 이제는 계절로도 ...",89,2024.01.03,대학 - 입학 및 학사운영,0,0
8,7795,마이유레카 모바일 웹 버전 개선 부탁드립니다,100,2023.12.05,기타,4,0
9,7794,법정의무교육 수강에 대한 제도 구축을 건의합니다,64,2023.12.01,행정지원 - 학교관련 제도,2,0


- 필요없는 칼럼 drop 및 숫자형 데이터 변환

In [90]:
petition_df.drop('NO', axis=1, inplace=True)
petition_df.drop('등록일', axis=1, inplace=True)

In [91]:
petition_df['조회수'] = pd.to_numeric(petition_df['조회수'], errors='coerce')
petition_df['공감'] = pd.to_numeric(petition_df['공감'], errors='coerce')
petition_df['비공감'] = pd.to_numeric(petition_df['비공감'], errors='coerce')

In [92]:
petition_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1843 entries, 0 to 1842
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제목      1843 non-null   object
 1   조회수     1843 non-null   int64 
 2   카테고리    1843 non-null   object
 3   공감      1843 non-null   int64 
 4   비공감     1843 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 72.1+ KB


## 2. 데이터 전처리

- 정규표현식 모듈을 사용해 훈련에 방해가 되는 문자 제거

In [93]:
# 숫자 제거

import re

petition_df = petition_df.fillna(' ')
petition_df['제목'] = petition_df['제목'].apply(lambda x : re.sub(r"\d+", " ", x))

In [94]:
# 영어 제거
def remove_english(text):
    result = re.sub(r'[a-zA-Z]+', '', text)
    return result

petition_df['제목'] = petition_df['제목'].apply(remove_english)

In [95]:
# 한자 제거
def remove_chinese_chars_from_df(df, column_name):
    pattern = re.compile('[\u4e00-\u9fff]+')
    df[column_name] = df[column_name].apply(lambda x: pattern.sub('', x))
    return df

petition_df = remove_chinese_chars_from_df(petition_df, '제목')

In [96]:
# 특수문자 제거 함수 정의
def remove_special_chars(text):
    return text.translate(str.maketrans('', '', '#$%^&*~-+=?!'))

# '제목' 열에만 특수문자 제거 함수 적용
petition_df['제목'] = petition_df['제목'].apply(lambda x: remove_special_chars(x) if isinstance(x, str) else x)

In [97]:
#null 행 제거
petition_df.dropna(subset=['제목'], inplace=True)
petition_df = petition_df[petition_df['제목'].str.strip() != '']

In [98]:
petition_df

,제목,조회수,카테고리,공감,비공감
0,북한학과 박사 논문 연구계획서 불합격 제도 시정 요구,72,대학,0,0
1,사설강의 '인프런'과 대학 제휴 협약을 맺어주세요,172,학생지원 - 학술지원,28,0
2,학점 포기제 일부 허용 건의,9,행정지원 - 학교관련 제도,0,0
3,편입학 제도 개선 관련 문의,2,행정지원 - 학교관련 제도,0,0
4,기숙사 통금 시간 연장 건의,1013,생활지원 - 기숙사,286,2
...,...,...,...,...,...
1838,수강시뮬레이션이나 졸업시뮬레이션 항상 볼 수 있게 해주세요.,2,기타,0,0
1839,편입학 수강신청,12,행정지원 - 학사운영 전반에 관한 업무,0,0
1840,학교에서 본교생을 대상으로 운영하고 있는 프로그램에 교수 및 교직원 자녀가 사적으로...,13,기타,0,0
1841,의 세탁소,6,생활지원 - 기숙사,0,0


- 칼럼 제목 변경

In [99]:
df = petition_df.copy()
df.rename(columns={'제목': 'title'}, inplace=True)

In [100]:
df.rename(columns={'조회수': 'views'}, inplace=True)
df.rename(columns={'등록일': 'date'}, inplace=True)
df.rename(columns={'공감': 'agree'}, inplace=True)
df.rename(columns={'비공감': 'disagree'}, inplace=True)
df.rename(columns={'카테고리': 'category'}, inplace=True)

In [101]:
df.describe()

,views,agree,disagree
count,1843.000000,1843.000000,1843.000000
mean,26.805209,3.064569,0.051004
std,94.373576,23.742298,0.376571
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000
75%,12.000000,0.000000,0.000000
max,1530.000000,473.000000,7.000000


## 3. 청원 통과 확률 모델링

100명 도달시 청원 통과를 기준으로 청원 통과 확률 부여

In [102]:
# 'probability' 칼럼 생성
df['probability'] = df['agree'].apply(lambda x: x if x < 100 else 100)

In [103]:
print(df[['category', 'agree', 'disagree', 'probability']])

                   category  agree  disagree  probability
0                        대학      0         0            0
1               학생지원 - 학술지원     28         0           28
2            행정지원 - 학교관련 제도      0         0            0
3            행정지원 - 학교관련 제도      0         0            0
4                생활지원 - 기숙사    286         2          100
...                     ...    ...       ...          ...
1838                     기타      0         0            0
1839  행정지원 - 학사운영 전반에 관한 업무      0         0            0
1840                     기타      0         0            0
1841             생활지원 - 기숙사      0         0            0
1842                     기타      0         0            0

[1843 rows x 4 columns]


- 라벨 인코더

In [105]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd


# LabelEncoder 인스턴스 생성
label_encoder = LabelEncoder()

# 카테고리 피처를 숫자로 인코딩
df['category'] = label_encoder.fit_transform(df['category'])

In [106]:
df['category']

0        1
1       19
2       31
3       31
4        5
        ..
1838     0
1839    32
1840     0
1841     5
1842     0
Name: category, Length: 1843, dtype: int64

In [107]:
df

,title,views,category,agree,disagree,probability
0,북한학과 박사 논문 연구계획서 불합격 제도 시정 요구,72,1,0,0,0
1,사설강의 '인프런'과 대학 제휴 협약을 맺어주세요,172,19,28,0,28
2,학점 포기제 일부 허용 건의,9,31,0,0,0
3,편입학 제도 개선 관련 문의,2,31,0,0,0
4,기숙사 통금 시간 연장 건의,1013,5,286,2,100
...,...,...,...,...,...,...
1838,수강시뮬레이션이나 졸업시뮬레이션 항상 볼 수 있게 해주세요.,2,0,0,0,0
1839,편입학 수강신청,12,32,0,0,0
1840,학교에서 본교생을 대상으로 운영하고 있는 프로그램에 교수 및 교직원 자녀가 사적으로...,13,0,0,0,0
1841,의 세탁소,6,5,0,0,0


In [108]:
#예측에 사용되지 않는 title feature drop
df.drop('title', axis=1, inplace=True) 

## 2. EDA

- 전처리

In [109]:
from sklearn.preprocessing import StandardScaler

# StandardScaler 객체 생성
scaler = StandardScaler()

# 조회수 데이터 스케일링
df['views'] = scaler.fit_transform(df[['views']])

In [110]:
# StandardScaler 객체 생성
scaler = StandardScaler()

# 조회수 데이터 스케일링
df['disagree'] = scaler.fit_transform(df[['disagree']])

## 5. 모델 훈련

In [111]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
import numpy as np

import pandas as pd
from sklearn.model_selection import train_test_split

# 가정: 'data'는 pd.DataFrame이며, 여러분의 데이터를 이미 포함하고 있다.
# 'probability'를 타겟으로 설정하고, 나머지는 피처로 사용합니다.
X = df.drop('probability', axis=1)
y = df['probability']

# 데이터를 훈련 세트와 테스트 세트로 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 회귀 모델 생성 및 훈련
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# 테스트 데이터에 대해 예측
y_pred = model.predict(X_test)

# 예측 결과를 0~100 사이로 제한
y_pred_clipped = np.clip(y_pred, 0, 100)

In [55]:
#예측 결과를 정수형으로 변환
y_pred_rounded = np.round(y_pred).astype(int)

### R2 Score 평가

In [112]:
from sklearn.metrics import r2_score

# 결정 계수(R^2) 계산
r2 = r2_score(y_test, y_pred)
print(f'결정 계수(R^2): {r2:.3f}')

결정 계수(R^2): 0.990


#### 예측 결과 출력

In [57]:
result = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred_rounded})
result.to_csv('result_t2.csv', index = False)